
# Using the serial communication utils
### This example shows how to use the BaslerSerial class to communicate with a serial device connected to a Basler camera.

In [1]:
import time

import serial
from pypylon import pylon as py
from pypylon_contrib.serial_communication import BaslerSerial

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

Create a BaslerSerial instance by passing the camera instance and the serial port parameters.

In [2]:

my_camera = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice())

cam_serial = BaslerSerial(camera=my_camera,
                          rx_line="Line3",
                          tx_line="Line2",
                          baudrate=115200,
                          bytesize=8,
                          parity=serial.PARITY_NONE,
                          timeout=1,)

Create a Serial instance by passing port name and the serial port parameters.


In [3]:

port_serial = serial.Serial("COM7",
                            baudrate=115200,
                            bytesize=8,
                            parity=serial.PARITY_NONE,
                            timeout=1)

In [4]:
# Both serial ports have nearly the same API, so you can use them interchangeably and replace a standard serial.Serial instance with a BaslerSerial.
print(cam_serial)
print(port_serial)

BaslerSerial<id=0x232599eb760, open=True>(port='Basler_40110957', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)
Serial<id=0x2325ab85150, open=True>(port='COM7', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)


### Writing a message via the camera serial port and read it back from the serial device.
The amount of write data is unlimited, due to host side buffering,
but if the message is larger than the camera's tx buffer, the write will block until the message is sent completely.

In [5]:

test_message_write = b"Sending a test message from Camera to serial device, without limitation due to host side buffer"

# the write will block if the test message is larger than the camera tx buffer,
# but to ensure that the message is sent always complete, add an additional flush after the write
bytes_written = cam_serial.write(test_message_write)
cam_serial.flush()
# for really low baudrates, the last byte may be in the output register of the camera,
# so we wait for an additional 5ms to ensure that the message is sent completely (~1 byte at 2400 Baud)
time.sleep(0.005)

bytes_waiting = port_serial.in_waiting
print(f"Bytes written: {bytes_written}")
print(f"Bytes waiting: {bytes_waiting}")
test_message_read = port_serial.read(bytes_waiting)
print(f"Send message    : {test_message_write}")
print(f"Received message: {test_message_read}")

# check the status of the camera serial port
print(f"Camera Status: {cam_serial.check_status(assert_ok=False)}")

Bytes written: 95
Bytes waiting: 95
Send message    : b'Sending a test message from Camera to serial device, without limitation due to host side buffer'
Received message: b'Sending a test message from Camera to serial device, without limitation due to host side buffer'
Camera Status: []


### Writing a message via the serial device and read it back from the camera port.
The amount of read data is limited, if no thread is calling "receive" periodically to pull the data from the camera.
If the message is larger than the camera's rx buffer, the read will be incomplete and the camera will report an overflow error.

In [6]:


rx_limit = my_camera.BslSerialTransferLength.Max
too_long_message = ("-" * rx_limit + " Overflow!").encode()

test_message_write = b"Sending message" # normal size message, no overflow expected
# test_message_write = too_long_message # message size exceeds limitations, overflow expected


bytes_written = port_serial.write(test_message_write)
port_serial.flush()

# the buffer inside the com-device may hold more bytes, lets wait for additional 0.1 seconds
time.sleep(0.1)

bytes_waiting = cam_serial.in_waiting
print(f"Bytes written: {bytes_written}")
print(f"Bytes waiting: {bytes_waiting}")
test_message_read = cam_serial.read(bytes_waiting)
print(f"Send message    : {test_message_write}")
print(f"Received message: {test_message_read}")

# check the status of the camera serial port, due to the limitation of the camera rx buffer,
# this could be "RX_FIFO_OVERFLOW" if len(test_message_write) > my_camera.camera.BslSerialTransferLength.Max
print(f"Camera Status: {cam_serial.check_status(assert_ok=False)}")

Bytes written: 15
Bytes waiting: 15
Send message    : b'Sending message'
Received message: b'Sending message'
Camera Status: []


In [7]:
# Close the serial ports and the camera.
cam_serial.close()  # Warning: closing the camera serial port will automatically close the camera, only marks the serial port as closed
my_camera.Close()  # Warning: closing the camera will not automatically mark the serial port as closed, if BaslerSerial instance is still open

port_serial.close()